Imports

In [1]:
import random
import operator

import numpy as np
import matplotlib.pyplot as plt

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

Create Classes

In [2]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

Primitives and Toolbox

In [3]:
random.seed(0)

def identity(x):
    return x

pset = gp.PrimitiveSetTyped("MAIN", in_types=[float, float, float, float, float, float, float], ret_type=bool)
pset.addPrimitive(np.less, in_types=[float, float], ret_type=bool)
pset.addPrimitive(np.greater, in_types=[float, float], ret_type=bool)
pset.addPrimitive(np.equal, in_types=[float, float], ret_type=bool)
pset.addPrimitive(np.logical_not, in_types=[bool], ret_type=bool)
pset.addPrimitive(np.logical_and, in_types=[bool, bool], ret_type=bool)
pset.addPrimitive(np.logical_or, in_types=[bool, bool], ret_type=bool)
pset.addPrimitive(identity, in_types=[float], ret_type=float)
pset.addEphemeralConstant("float_const", ephemeral=lambda: random.randint(0, 100), ret_type=float)
pset.addEphemeralConstant("bool_const", ephemeral=lambda: random.randint(0, 1), ret_type=bool)

pset.renameArguments(ARG0="Pclass")
pset.renameArguments(ARG1="Sex")
pset.renameArguments(ARG2="Age")
pset.renameArguments(ARG3="SibSp")
pset.renameArguments(ARG4="Parch")
pset.renameArguments(ARG5="Fare")
pset.renameArguments(ARG6="Embarked")

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

Data

In [ ]:
# Load data into numpy array (numeric data only)
sex = lambda x: 0.0 if x == b"male" else 1.0  # male = 0, female = 1
embarked = lambda x: 0.0 if x == b'C' else 2.0 if x == b'Q' else 1.0 # C = 0, S = 1, Q = 2; fill missing embarked values with S
data = np.genfromtxt(open("../data/train.csv"), delimiter=',', skip_header=1, usecols=(1, 2, 5, 6, 7, 8, 10, 12), converters={5: sex, 12: embarked})

# Fill missing age and fare (testing data only) values with averages for each class
def fill_missing(data):
    class1 = data[:, 0] == 1
    class2 = data[:, 0] == 2
    class3 = data[:, 0] == 3

    avg_age1 = np.nanmean(data[class1, 2])
    avg_age2 = np.nanmean(data[class2, 2])
    avg_age3 = np.nanmean(data[class3, 2])
    avg_fare1 = np.nanmean(data[class1, 5])
    avg_fare2 = np.nanmean(data[class2, 5])
    avg_fare3 = np.nanmean(data[class3, 5])

    age_nans = np.isnan(data[:, 2])
    fare_nans = np.isnan(data[:, 5])

    data[age_nans & class1, 2] = avg_age1
    data[age_nans & class2, 2] = avg_age2
    data[age_nans & class3, 2] = avg_age3
    data[fare_nans & class1, 5] = avg_fare1
    data[fare_nans & class2, 5] = avg_fare2
    data[fare_nans & class3, 5] = avg_fare3

X = data[:, 1:]
y = data[:, 0]
fill_missing(X)

Evaluation Function

In [ ]:
def evalClassif(individual, examples, labels, pset):
    func = gp.compile(expr=individual, pset=pset)
    preds = func(*examples.T)

    return (np.sum((preds == 1) & (labels == 0)), np.sum((preds == 0) & (labels == 1)))

toolbox.register("evaluate", evalClassif, examples=X, labels=y, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

Main Algorithm

In [ ]:
NGEN = 250
MU = 500
LAMBDA = 1000
CXPB = 0.2
MUTPB = 0.5

class ParetoFrontAUC(tools.ParetoFront):
    def __init__(self):
        super().__init__()
        self.aucs = []
    
    def update(self, pop):
        super().update(pop)
        fitness_1 = [ind.fitness.values[0] for ind in self]
        fitness_2 = [ind.fitness.values[1] for ind in self]
        auc = np.sum(np.abs(np.diff(fitness_1))*fitness_2[:-1])
        self.aucs.append(auc)

pop = toolbox.population(n=MU)
hof = ParetoFrontAUC()
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

pop, logbook = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                          halloffame=hof)

gen	nevals	avg              	std                        	min    	max        
0  	500   	[218.846 205.29 ]	[253.4849942  152.25367615]	[0. 0.]	[549. 342.]
1  	689   	[ 31.754 320.89 ]	[114.35464785  67.64618171]	[0. 0.]	[549. 342.]
2  	697   	[  0.856 340.578]	[15.61048571 10.34658958]  	[  0. 179.]	[347. 342.]
3  	698   	[1.24000e-01 3.40788e+02]	[1.8784632  3.78510977]    	[  0. 280.]	[ 39. 342.]
4  	715   	[7.8000e-02 3.4019e+02]  	[1.74238802 3.1083597 ]    	[  0. 280.]	[ 39. 342.]
5  	694   	[  0.864 339.308]        	[18.36947207  6.27225127]  	[  0. 235.]	[411. 342.]


Plots

In [ ]:
print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))
gen, avg, min_, max_ = logbook.select("gen", "avg", "min", "max")
plt.plot(gen, avg, label="average")
plt.plot(gen, min_, label="minimum")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="upper left")
plt.show()

AUC Graph

In [ ]:
gen = logbook.select("gen")
plt.plot(gen, hof.aucs)
plt.xlabel("Generation")
plt.ylabel("AUC")
plt.title("Praeto Front Progression")
plt.show()

Praeto Front

In [ ]:
"""Split fitness values into separate lists"""
fitness_1 = [ind.fitness.values[0] for ind in hof]
fitness_2 = [ind.fitness.values[1] for ind in hof]
pop_1 = [ind.fitness.values[0] for ind in pop]
pop_2 = [ind.fitness.values[1] for ind in pop]

'''Print dominated population for debugging'''
# for ind in pop:
#     print(ind.fitness)

plt.scatter(pop_1, pop_2, color='b')
plt.scatter(fitness_1, fitness_2, color='r')
plt.plot(fitness_1, fitness_2, color='r', drawstyle='steps-post')
plt.xlabel("False Negatives")
plt.ylabel("False Positives")
plt.title("Pareto Front")
plt.show()

f1 = np.array(fitness_1)
f2 = np.array(fitness_2)

"""Calculate area under curve with least squares method"""
print("Area Under Curve: %s" % (np.sum(np.abs(np.diff(f1))*f2[:-1])))